# Customer analysis

In this exercise, you work for a consulting firm as data analyst. One of your clients asked for help to understand its customer buying behavior. 

You dispose of a dataset from its e-commerce website with 13 features

- `order_id` : ID if the order
- order_status : Status of the order
- order_purchase_timestamp : Timestamp of the purchase
- order_delivered_customer_date : Timestamp of the delivery to the customer
- payment_type : Method of payment
- payment_installments : Number of payments for this purchase (does NOT affect the payment value)
- payment_value : Total amount of the payment
- review_score : Score given by the client, ranges from 1 to 5
- customer_unique_id : ID of the customer who purchased
- customer_zip_code_prefix : Zip code of the customer
- customer_city : City of the customer
- customer_state : State of the customer
- product_category_name_english : Name of the product category in English

With this data, you have to analyze customer behavior with two techniques:

- Customer segmentation with RFM
- Cohort analysis from Sept 2016 to October 2018.

At the end, you have to present your analysis with actionable insights to improve the customer relationship of your client. 

# Understanding RFM Segmentation 

❓ What are the 3 metrics used in an RFM Segmentation? (hint : What does RFM stand for?) 

> RFM Segmentation metrics are recency, frequency, and monetary value

❓ Can you summarize what each metric measures in one sentence?

> Recency measures customer activity (time since last order), frequency defines transactions/visits regularity and monetary metric points out customer's purchasing power.

❓ What data / or type of data do you need for each metric ? 

> Recency will need purchases inherent data as well as their data time, to measure frequency metric we'll need the same type of data as for recency and to define monetary or purchasing power we'll need transactions value in addition of all inherent data.

Let's explore the dataset!

# Exploratory Data Analysis

## Importing libraries

In [67]:
from datetime import timedelta

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

Load the dataset and start an Exploratory Data Analysis

In [2]:
df = pd.read_csv('../data/db_customers.csv')
df

,order_id,order_status,order_purchase_timestamp,order_delivered_customer_date,payment_type,payment_installments,payment_value,review_score,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state,product_category_name_english
0,e481f51cbdc54678b7cc49136f2d6af7,delivered,2017-10-02 10:56:33,2017-10-10 21:25:13,credit_card,1.0,18.12,4,7c396fd4830fd04220f754e42b4e5bff,3149,sao paulo,SP,housewares
1,e481f51cbdc54678b7cc49136f2d6af7,delivered,2017-10-02 10:56:33,2017-10-10 21:25:13,voucher,1.0,2.00,4,7c396fd4830fd04220f754e42b4e5bff,3149,sao paulo,SP,housewares
2,e481f51cbdc54678b7cc49136f2d6af7,delivered,2017-10-02 10:56:33,2017-10-10 21:25:13,voucher,1.0,18.59,4,7c396fd4830fd04220f754e42b4e5bff,3149,sao paulo,SP,housewares
3,53cdb2fc8bc7dce0b6741e2150273451,delivered,2018-07-24 20:41:37,2018-08-07 15:27:45,boleto,1.0,141.46,4,af07308b275d755c9edb36a90c618231,47813,barreiras,BA,perfumery
4,47770eb9100c2d0c44946d9cf07ec65d,delivered,2018-08-08 08:38:49,2018-08-17 18:06:29,credit_card,3.0,179.12,5,3a653a41f6f9fc3d2a113cf8398680e8,75265,vianopolis,GO,auto
...,...,...,...,...,...,...,...,...,...,...,...,...,...
105378,9c5dedf39a927c1b2549525ed64a053c,delivered,2017-03-09 09:54:05,2017-03-17 15:08:01,credit_card,3.0,85.08,5,6359f309b166b0196dbf7ad2ac62bb5a,12209,sao jose dos campos,SP,health_beauty
105379,63943bddc261676b46f01ca7ac2f7bd8,delivered,2018-02-06 12:58:58,2018-02-28 17:37:56,credit_card,3.0,195.00,4,da62f9e57a76d978d02ab5362c509660,11722,praia grande,SP,baby
105380,83c1379a015df1e13d02aae0204711ab,delivered,2017-08-27 14:46:43,2017-09-21 11:24:17,credit_card,5.0,271.01,5,737520a9aad80b3fbbdad19b66b37b30,45920,nova vicosa,BA,home_appliances_2
105381,11c177c8e97725db2631073c19f07b62,delivered,2018-01-08 21:28:27,2018-01-25 23:32:54,credit_card,4.0,441.16,2,5097a5312c8b157bb7be58ae360ef43c,28685,japuiba,RJ,computers_accessories


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 105383 entries, 0 to 105382
Data columns (total 13 columns):
 #   Column                         Non-Null Count   Dtype  
---  ------                         --------------   -----  
 0   order_id                       105383 non-null  object 
 1   order_status                   105383 non-null  object 
 2   order_purchase_timestamp       105383 non-null  object 
 3   order_delivered_customer_date  102224 non-null  object 
 4   payment_type                   105382 non-null  object 
 5   payment_installments           105382 non-null  float64
 6   payment_value                  105382 non-null  float64
 7   review_score                   105383 non-null  int64  
 8   customer_unique_id             105383 non-null  object 
 9   customer_zip_code_prefix       105383 non-null  int64  
 10  customer_city                  105383 non-null  object 
 11  customer_state                 105383 non-null  object 
 12  product_category_name_english 

❓ Which features should be formated in datetime ? 
Convert them

> The features which should be formatted in date time are order_purchase_timestamp and order_delivered_customer_date

❓ Which features are Categorical ? List the unique values on these features.

> The categorical features are order_status, payment_type, review_score, customer_zip_code_prefix, customer_city, customer_state and product_category_name_english

In [6]:
ctg_features = ['order_status','payment_type', 'review_score',
                'customer_zip_code_prefix', 'customer_city', 'customer_state',
                'product_category_name_english']

for col in ctg_features:
    print(df[col].unique())

['delivered' 'invoiced' 'shipped' 'processing' 'unavailable' 'canceled'
 'created' 'approved']
['credit_card' 'voucher' 'boleto' 'debit_card' 'not_defined' nan]
[4 5 2 1 3]
[ 3149 47813 75265 ... 83870  5127 45920]
['sao paulo' 'barreiras' 'vianopolis' ... 'messias targino'
 'campo do tenente' 'nova vicosa']
['SP' 'BA' 'GO' 'RN' 'PR' 'RS' 'RJ' 'MG' 'SC' 'RR' 'PE' 'TO' 'CE' 'DF'
 'SE' 'MT' 'PB' 'PA' 'RO' 'ES' 'AP' 'MS' 'MA' 'PI' 'AL' 'AC' 'AM']
['housewares' 'perfumery' 'auto' 'pet_shop' 'stationery' nan
 'furniture_decor' 'office_furniture' 'garden_tools'
 'computers_accessories' 'bed_bath_table' 'toys'
 'construction_tools_construction' 'telephony' 'health_beauty'
 'electronics' 'baby' 'cool_stuff' 'watches_gifts' 'air_conditioning'
 'sports_leisure' 'books_general_interest' 'small_appliances' 'food'
 'luggage_accessories' 'fashion_underwear_beach' 'christmas_supplies'
 'fashion_bags_accessories' 'musical_instruments'
 'construction_tools_lights' 'books_technical' 'costruction_tools_g

❓ Check for missing values. What do you notice ?


In [7]:
missing_values = df.isnull().sum()
missing_values

order_id                            0
order_status                        0
order_purchase_timestamp            0
order_delivered_customer_date    3159
payment_type                        1
payment_installments                1
payment_value                       1
review_score                        0
customer_unique_id                  0
customer_zip_code_prefix            0
customer_city                       0
customer_state                      0
product_category_name_english    2410
dtype: int64

> We could notice there are 3159 missing values in order_delivered_customer_date feature and 2410 ones in product_category_name_english.


❓ Can you find an explanation for the missing `order_delivered_customer_date` ?
(hint : Check for non delivered orders)

> In order_delivered_customer_date feature, we can suppose there are no date time since no product has still been delivered to the customer.

In [8]:
orders_delivered = (df['order_status'] != 'delivered').value_counts()
orders_delivered

False    102225
True       3158
Name: order_status, dtype: int64

> As we could see, there are 3158 orders which are not in 'delivered' status, so it's totally fine, even if there is one product who's missing.

❓ What are the proportions of each `order_status` ? 

In [9]:
orders_status = df.order_status.value_counts()/df.shape[0]*100
orders_status

delivered      97.003312
shipped         1.119725
canceled        0.634827
unavailable     0.617747
invoiced        0.313143
processing      0.304603
created         0.004745
approved        0.001898
Name: order_status, dtype: float64

❓ Which `order_status` do you think should be removed from the analysis

> We'll keep delivered and shipped status

In [10]:
orders_status = ['delivered', 'shipped']

❓ What feature would you choose in the e-commerce dataset provided? 

In [11]:
df.columns

Index(['order_id', 'order_status', 'order_purchase_timestamp',
       'order_delivered_customer_date', 'payment_type', 'payment_installments',
       'payment_value', 'review_score', 'customer_unique_id',
       'customer_zip_code_prefix', 'customer_city', 'customer_state',
       'product_category_name_english'],
      dtype='object')

> We'll choose the ones who refers to RFM, which are order_purchase_timestamp, payment_value & order_id as well as customer_unique_id

In [29]:
X = df[['order_purchase_timestamp', 'payment_value', 'order_id', 'customer_unique_id']]
X

,order_purchase_timestamp,payment_value,order_id,customer_unique_id
0,2017-10-02 10:56:33,18.12,e481f51cbdc54678b7cc49136f2d6af7,7c396fd4830fd04220f754e42b4e5bff
1,2017-10-02 10:56:33,2.00,e481f51cbdc54678b7cc49136f2d6af7,7c396fd4830fd04220f754e42b4e5bff
2,2017-10-02 10:56:33,18.59,e481f51cbdc54678b7cc49136f2d6af7,7c396fd4830fd04220f754e42b4e5bff
3,2018-07-24 20:41:37,141.46,53cdb2fc8bc7dce0b6741e2150273451,af07308b275d755c9edb36a90c618231
4,2018-08-08 08:38:49,179.12,47770eb9100c2d0c44946d9cf07ec65d,3a653a41f6f9fc3d2a113cf8398680e8
...,...,...,...,...
105378,2017-03-09 09:54:05,85.08,9c5dedf39a927c1b2549525ed64a053c,6359f309b166b0196dbf7ad2ac62bb5a
105379,2018-02-06 12:58:58,195.00,63943bddc261676b46f01ca7ac2f7bd8,da62f9e57a76d978d02ab5362c509660
105380,2017-08-27 14:46:43,271.01,83c1379a015df1e13d02aae0204711ab,737520a9aad80b3fbbdad19b66b37b30
105381,2018-01-08 21:28:27,441.16,11c177c8e97725db2631073c19f07b62,5097a5312c8b157bb7be58ae360ef43c


❓ What method would you use to group the data? 

> We could use 'group by' method to group the selected data.

❓ What feature would you use to group the data? 

> We'll use 'order_purchase_timestamp', 'payment_value', 'order_id' and 'customer_unique_id' feature.

Group the data. (Hint: you can carry out the grouping one by one and then merge, or even better use the .agg method and get a robust grouping in one line of code)

In [54]:
customers_df = X.groupby('customer_unique_id').agg(frequency = ('order_id', 'count'),
                                                   last_purchase_datetime = ('order_purchase_timestamp', 'max'),
                                                   monetary = ('payment_value', 'sum'))
customers_df

,frequency,last_purchase_datetime,monetary
customer_unique_id,,,
0000366f3b9a7992bf8c76cfdf3221e2,1,2018-05-10 10:56:27,141.90
0000b849f77a49e4a4ce2b2a4ca5be3f,1,2018-05-07 11:11:27,27.19
0000f46a3911fa3c0805444483337064,1,2017-03-10 21:05:03,86.22
0000f6ccb0745a6a4b88665a16c9f078,1,2017-10-12 20:29:41,43.62
0004aac84e0df4da2b147fca70cf8255,1,2017-11-14 19:45:42,196.89
...,...,...,...
fffcf5a5ff07b0908bd4e2dbc735a684,1,2017-06-08 21:00:36,2067.42
fffea47cd6d3cc0a88bd621562a9d061,1,2017-12-10 20:07:56,84.58
ffff371b4d645b6ecea244b27531430a,1,2017-02-07 15:49:16,112.46


What type of data is the R supposed to be? Is it in the right format? (Hint : investigate the difference between `datetime` and `timedelta`) 

> Recency data type is supposed to be 'datetime', in this case it should be 'timedelta' type, which represents the difference between two dates or times.

So there is one more step to carry out before the data is ready : Calculate the recency base on the date of the purchase

❓ What is the date of the least purchase?

In [55]:
customers_df.last_purchase_datetime

customer_unique_id
0000366f3b9a7992bf8c76cfdf3221e2    2018-05-10 10:56:27
0000b849f77a49e4a4ce2b2a4ca5be3f    2018-05-07 11:11:27
0000f46a3911fa3c0805444483337064    2017-03-10 21:05:03
0000f6ccb0745a6a4b88665a16c9f078    2017-10-12 20:29:41
0004aac84e0df4da2b147fca70cf8255    2017-11-14 19:45:42
                                           ...         
fffcf5a5ff07b0908bd4e2dbc735a684    2017-06-08 21:00:36
fffea47cd6d3cc0a88bd621562a9d061    2017-12-10 20:07:56
ffff371b4d645b6ecea244b27531430a    2017-02-07 15:49:16
ffff5962728ec6157033ef9805bacc48    2018-05-02 15:17:41
ffffd2657e2aad2907e67c3e9daecbeb    2017-05-02 20:18:45
Name: last_purchase_datetime, Length: 96096, dtype: object

❓How would you calculate the recency? (hint: what date would you use to compare with the purchase date? Today? probably not...)

> I would use the last day of the same year where customer made his last purchase, in this case we'll take their last purchase date and subtract it w/ the last day of the year when this transaction has been made or use the most recent transaction date ever as a reference to calculate days past w/out any transaction passed from this date.

Create a feature calculating the difference between `the most recent purchased + 1 day`, and `order_purchase_timestamp`.

In [74]:
# Convert InvoiceDate from object to datetime format
customers_df['last_purchase_datetime'] = pd.to_datetime(customers_df.last_purchase_datetime)

# Create snapshot date + 1 day
snapshot_date = customers_df.last_purchase_datetime.max() + timedelta(days=1)
print(snapshot_date)

2018-10-18 17:30:18


In [75]:
for i in customers_df.last_purchase_datetime:
    recency = (snapshot_date - i.max()).days

TypeError: 'Timestamp' object is not callable

Now that you have created a new dataset with the right columns, explore it to understand what you created.

❓Which column is the R? The F? The M?

Now for each metric independantly, build clutsers (Hint 1: you can use `.describe()` to have insights about each metrics. Hint 2: Don't choose more than 3 clusters)

Now that you have clusters for each category, explain the different buying behaviors

# Working with cohorts

❓What is a cohort? 

❓Why is it interesting to analyse cohorts ? 

❓What data (already existing in the original dataset) do we need to build a cohort? 

Build a new df with just these 3 features

Using the `pandas.Serie.dt.to_period()` methode, create a new feature converting `order_purchase_timestamp` in months

Create a new feature called `cohort` by grouping the previous df by `customer_unique_id`, and tranforming it's minimum value in months (with the same `pandas.Serie.dt.to_period()` as before)

In [ ]:
# For the teachers, To filter by only the customer that have done 2 or more purchases
df_more =pd.DataFrame(df_c.groupby('customer_unique_id').size()[df_c.groupby('customer_unique_id').size() > 1])
df_c = df_c.merge(df_more, how='left', on='customer_unique_id').rename(columns={0:'na'},inplace=True)

df_c = df_c[df_c['na'].notna()]

In [ ]:
df_c['cohort'] = df_c.groupby('customer_unique_id')['order_purchase_timestamp'] \
                 .transform('min') \
                 .dt.to_period('M') 
df_c

Now groupby `['cohort', 'order_month']` and aggregate by `nunique()` `customer_unique_id`

Finally, create a feature called `['period_number']` where you substract the `cohort` to the `order_month` and get the `'n'` attribute thanks to the `operator.attrgetter` method

Explore a cohort (for example 2017-01) and try to understand what it shows

Pivot the DataFrame using the methode : `DataFrame.pivot_table` (hint : use `'cohort'` as `index`, `'period_number'` as columns, and `'n_customers'` as `values`)

Now divide all the values (except you column 0) by the column 0 and name this new DataFrame `retention_matrix`

Plot your `retention matrix` using the code below : 

In [ ]:

with sns.axes_style("white"):
    fig, ax = plt.subplots(1, 2, figsize=(12, 8), sharey=True, gridspec_kw={'width_ratios': [1, 11]})
    
    # retention matrix
    sns.heatmap(retention_matrix, 
                mask=retention_matrix.isnull(), 
                annot=True, 
                fmt='.0%', 
                cmap='RdYlGn', 
                ax=ax[1])
    ax[1].set_title('Monthly Cohorts: User Retention', fontsize=16)
    ax[1].set(xlabel='# of periods',
              ylabel='')

    # cohort size
    cohort_size_df = pd.DataFrame(cohort_size).rename(columns={0: 'cohort_size'})
    white_cmap = mcolors.ListedColormap(['white'])
    sns.heatmap(cohort_size_df, 
                annot=True, 
                cbar=False, 
                fmt='g', 
                cmap=white_cmap, 
                ax=ax[0])

    fig.tight_layout()


What do you notice? 

How could we have know that before? (hint : Calculate the percentage of customers who have ordered more than once)

Here is the code for the analysis of customers who have bought more than once : 

In [ ]:
#start from your df created doing the RFM segmentation
df_c = df[['customer_unique_id', 'order_id',
           'order_purchase_timestamp']].drop_duplicates()

df_c['order_month'] = df_c['order_purchase_timestamp'].dt.to_period('M')

df_more = pd.DataFrame(df_c.groupby('customer_unique_id').size()[
                       df_c.groupby('customer_unique_id').size() > 1])

df_c = df_c.merge(df_more, how='left', on='customer_unique_id').rename(
    columns={0: 'na'})

df_c = df_c[df_c['na'].notna()]
df_c['cohort'] = df_c.groupby('customer_unique_id')['order_purchase_timestamp'] \
    .transform('min') \
    .dt.to_period('M')

df_cohort = df_c.groupby(['cohort', 'order_month']) \
    .agg(n_customers=('customer_unique_id', 'nunique')) \
    .reset_index(drop=False)
df_cohort['period_number'] = (
    df_cohort.order_month - df_cohort.cohort).apply(attrgetter('n'))
cohort_pivot = df_cohort.pivot_table(index='cohort',
                                     columns='period_number',
                                     values='n_customers')
cohort_size = cohort_pivot.iloc[:, 0]
retention_matrix = cohort_pivot.divide(cohort_size, axis=0)

with sns.axes_style("white"):
    fig, ax = plt.subplots(1, 2, figsize=(12, 8), sharey=True, gridspec_kw={
                           'width_ratios': [1, 11]})

    # retention matrix
    sns.heatmap(retention_matrix,
                mask=retention_matrix.isnull(),
                annot=True,
                fmt='.0%',
                cmap='RdYlGn',
                ax=ax[1])
    ax[1].set_title('Monthly Cohorts: User Retention', fontsize=16)
    ax[1].set(xlabel='# of periods',
              ylabel='')

    # cohort size
    cohort_size_df = pd.DataFrame(cohort_size).rename(
        columns={0: 'cohort_size'})
    white_cmap = mcolors.ListedColormap(['white'])
    sns.heatmap(cohort_size_df,
                annot=True,
                cbar=False,
                fmt='g',
                cmap=white_cmap,
                ax=ax[0])

    fig.tight_layout()